In [1]:
from elasticsearch import Elasticsearch

In [2]:
es = Elasticsearch(
    "https://localhost:9200",
    basic_auth=("elastic","LQym+efHnUy9DbT-jtD2"),
    ca_certs="/Users/Mohamed/elasticsearch-8.9.1/config/certs/http_ca.crt"
)
es.ping()

True

## Prepare the data

In [3]:
import pandas as pd

df = pd.read_csv("myntra_products_catalog.csv").loc[:499]
df.head()

JobID                Title                Company                Date  \
0  10017413   Software Engineer               Tech Co.       December 2023  
1  10016283  Senior Software Developer     InnovateTech        January 2023  
2  10009781  Full Stack Engineer      Digital Solutions Inc.    February 2023  
3  10015921  Backend Developer       NextGen Software            March 2023  
4  10017833  Frontend Engineer       VisionaryTech              April 2023  

                                 Description  
0  Looking for a skilled Software Engineer pro...  
1  Join our team as a Senior Software Develop...  
2  Seeking a Full Stack Engineer experienced...  
3  NextGen Software is hiring a Backend Deve...  
4  VisionaryTech seeks a Frontend Engineer w...

In [6]:
df.isna().value_counts()

JobTitle  Company  Description  Date  Qualifications  Responsibilities  location  JobID
False      False        False         False   False        False      False        False           500
Name: count, dtype: int64

In [5]:
df.fillna("None", inplace=True)

## Convert the relevant field to Vector using BERT model

In [7]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-mpnet-base-v2')

/Users/Mohamed/semantic_search/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
df["DescriptionVector"] = df["Description"].apply(lambda x: model.encode(x))

In [9]:
df.head()

,JobTitle,Company,Description,location,Date,Qualifications,Responsibilities,JobID,DescriptionVector
0,10017413,Software Developper...,DKNY,full stack developer proficient in java,december 2023,maintain codebase,null...,1526,"[0.027645726, -0.0026341523, -0.0035884595, 0...."


In [10]:
es.ping()

True

## Create new index in ElasticSearch!

In [12]:
from indexMapping import indexMapping

es.indices.create(index="all_jobss", mappings=indexMapping)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'all_jobs'})

## Ingest the data into index

In [14]:
record_list = df.to_dict("records")

In [16]:
for record in record_list:
    try:
        es.index(index="all_jobs", document=record, id=record["ProductID"])
    except Exception as e:
        print(e)

In [17]:
es.count(index="all_products")

ObjectApiResponse({'count': 500, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}})

## Search the data

In [19]:
input_keyword = "Blue Shoes"
vector_of_input_keyword = model.encode(input_keyword)

query = {
    "field" : "DescriptionVector",
    "query_vector" : vector_of_input_keyword,
    "k" : 2,
    "num_candidates" : 500, 
}

res = es.knn_search(index="all_products", knn=query , source=["ProductName","Description"])
res["hits"]["hits"]

/var/folders/3k/901cf0jd1lngqxfl_y527qp00000gn/T/ipykernel_22712/2784084207.py:11: ElasticsearchWarning: The kNN search API has been replaced by the `knn` option in the search API.
  res = es.knn_search(index="all_products", knn=query , source=["ProductName","Description"])


[{'_index': 'all_jobs',
  '_id': '10018013',
  '_score': 0.61429405,
  '_source': {'JobTitle': 'Senior Java Developer',
   'Description': 'A full stack web developer'}},
